# page 87 : 4Layer XOR with ReLU

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_data = np.loadtxt('data_xor.txt', unpack=True, dtype='float32')

In [3]:
x_data = np.transpose(train_data[0:-1])
y_data = np.reshape(train_data[-1:], (4, 1))

X = tf.placeholder(tf.float32, name='x-input')
Y = tf.placeholder(tf.float32, name='y-input')

In [4]:
with tf.name_scope('WEIGHT') as scope:
    W1 = tf.Variable(tf.random_uniform([2, 5], -1., 1.), name='weight1')
    W2 = tf.Variable(tf.random_uniform([5, 5], -1., 1.), name='weight2')
    W3 = tf.Variable(tf.random_uniform([5, 4], -1., 1.), name='weight3')
    W4 = tf.Variable(tf.random_uniform([4, 1], -1., 1.), name='weight4')
    
with tf.name_scope('BIAS') as scope:
    b1 = tf.Variable(tf.random_uniform([5], -1., 1.), name='bias1')
    b2 = tf.Variable(tf.random_uniform([5], -1., 1.), name='bias2')
    b3 = tf.Variable(tf.random_uniform([4], -1., 1.), name='bias3')
    b4 = tf.Variable(tf.random_uniform([1], -1., 1.), name='bias4')
    

In [5]:
with tf.name_scope('layer2') as scope:
    L2 = tf.nn.relu(tf.matmul(X, W1) + b1)
    
with tf.name_scope('layer3') as scope:
    L3 = tf.nn.relu(tf.matmul(L2, W2) + b2)
    
with tf.name_scope('layer4') as scope:
    L4 = tf.nn.relu(tf.matmul(L3, W3) + b3)
    
    

In [6]:
with tf.name_scope('hypothesis') as scope:
    hypothesis = tf.sigmoid(tf.matmul(L4, W4) + b4)
    

In [7]:
with tf.name_scope('cost') as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    cost_sum = tf.summary.scalar('cost', cost)

In [8]:
w1_hist_sum = tf.summary.histogram('weight1', W1)
w2_hist_sum = tf.summary.histogram('weight2', W2)
w3_hist_sum = tf.summary.histogram('weight3', W3)
w4_hist_sum = tf.summary.histogram('weight4', W4)

In [9]:
b1_hist_sum = tf.summary.histogram('bias1', b1)
b2_hist_sum = tf.summary.histogram('bias2', b2)
b3_hist_sum = tf.summary.histogram('bias3', b3)
b4_hist_sum = tf.summary.histogram('bias4', b4)

y_hist_sum = tf.summary.histogram('y', Y)

In [10]:
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(cost)

In [11]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)
    
    for step in range(2001):
        feed = {X: x_data, Y: y_data}
        sess.run(train, feed_dict=feed)
        if (step % 10 == 0):
            summary = sess.run(merged, feed_dict=feed)
            writer.add_summary(summary, step)     
            
    prediction = tf.equal(tf.floor(hypothesis + 0.5), Y)

    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    print (sess.run([hypothesis, tf.floor(hypothesis + 0.5), prediction], feed_dict=feed))
    print ('accuracy:', sess.run(accuracy, feed_dict=feed))

[array([[ 0.46461517],
       [ 0.50445962],
       [ 0.54583687],
       [ 0.45061421]], dtype=float32), array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.]], dtype=float32), array([[ True],
       [ True],
       [ True],
       [ True]], dtype=bool)]
accuracy: 1.0


# Tensorboard로 확인하기
## tensorboard --logdir=./logs --port 6006